In [55]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [56]:
from threading import Thread

class StreamingThread(Thread):
    def __init__(self, ssc):
        Thread.__init__(self)
        self.ssc = ssc
    def run(self):
        ssc.start()
        ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

In [57]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [58]:
spark

In [59]:
from pyspark.streaming import StreamingContext
from pyspark.sql.functions import udf, struct, array, col, lit
from pyspark.sql.types import IntegerType

In [60]:
data = sc.textFile('file:///C:/Users/lenne/Desktop/spark/coding_and_data/data/lots_of_data/tweets')
data.first()

'{"tweet_id": 1380150333767262211, "tweet_text": "#\\u2588\\u2588\\u2588\\u2588\\u2588\\u2588\\u2588 bubbles under surface....\\n\\n#\\u2588\\u2588\\u2588\\u2588\\u2588\\u2588\\u2588 Backs #\\u2588\\u2588\\u2588\\u2588\\u2588\\u2588\\u2588, but Some Call for a Clearer Warning to #\\u2588\\u2588\\u2588\\u2588\\u2588\\u2588\\u2588 https://t.co/56tmzwLaS0", "label": "#biden"}'

In [61]:
df = spark.read.json(data)
df.show()

+--------+-------------------+--------------------+
|   label|           tweet_id|          tweet_text|
+--------+-------------------+--------------------+
|  #biden|1380150333767262211|#███████ bubbles ...|
|  #biden|1380150113650274308|Nothing to see he...|
|#vaccine|1380150618275389448|Well that was qui...|
|#vaccine|1380150554974904321|Good morning, Twi...|
|#vaccine|1380150530526306314|Here for my secon...|
|#vaccine|1380150486339366928|You don't have a ...|
|#vaccine|1380150434208358402|Because only cert...|
|#vaccine|1380150386863013894|#███████
#███████...|
|#vaccine|1380150373399339009|Proud to see thou...|
|  #covid|1380150776299937795|The pandemic is n...|
|  #covid|1380150698948575232|Tandon's next Emp...|
|  #covid|1380150671123623942|2 days virtual Co...|
|  #covid|1380150657823469573|Toronto ICU doc: ...|
|  #covid|1380150622796742658|India records 4th...|
|  #covid|1380150957300969473|When #███████ hit...|
|  #covid|1380150954503381000|With low literacy...|
|  #covid|13

In [62]:
df.count()

1298

In [123]:
from pyspark.sql.functions import when,regexp_replace
df2 = df.withColumn("label", when(df.label == "#biden",1)
                                 .when(df.label == "#inflation",2)
                    .when(df.label == "#china",3)
                    .when(df.label == "#stopasianhate",4)
                    .when(df.label == "#covid",5)
                    .when(df.label == "#vaccine",6)
                                 .when(df.label.isNull() ,"")
                                 .otherwise(df.label))
df2 = df2.withColumn('tweet_text', regexp_replace('tweet_text', r'[#@][^\s#@]+', ''))
df2 = df2.withColumn('tweet_text', regexp_replace('tweet_text', '\n', ''))
df2 = df2.withColumn('tweet_text', regexp_replace('tweet_text', '  ', ''))
df2.show(20)

pd_df2 = df2.toPandas()
display(pd_df2)
print(pd_df2['label'].value_counts())

+-----+-------------------+--------------------+
|label|           tweet_id|          tweet_text|
+-----+-------------------+--------------------+
|    1|1380150333767262211| bubbles under su...|
|    1|1380150113650274308|Nothing to see he...|
|    6|1380150618275389448|Well that was qui...|
|    6|1380150554974904321|Good morning, Twi...|
|    6|1380150530526306314|Here for my secon...|
|    6|1380150486339366928|You don't have a ...|
|    6|1380150434208358402|Because only cert...|
|    6|1380150386863013894|Ocugen: Potential...|
|    6|1380150373399339009|Proud to see thou...|
|    5|1380150776299937795|The pandemic is n...|
|    5|1380150698948575232|Tandon's next Emp...|
|    5|1380150671123623942|2 days virtual Co...|
|    5|1380150657823469573|Toronto ICU doc: ...|
|    5|1380150622796742658|India records 4th...|
|    5|1380150957300969473|Whenhit,saw the n...|
|    5|1380150954503381000|With low literacy...|
|    5|1380150949264695304|Republican Lawmak...|
|    5|1380150948895

,label,tweet_id,tweet_text
0,1,1380150333767262211,bubbles under surface.... Backsbut Some Call ...
1,1,1380150113650274308,Nothing to see here...Five people killed ininc...
2,6,1380150618275389448,Well that was quick!! Poked by Pfizer... https...
3,6,1380150554974904321,"Good morning, Twitterverse. My kids are gettin..."
4,6,1380150530526306314,Here for my second jab and the place is hoppin...
...,...,...,...
1293,2,1392046806712922112,Inflation jitters... Nasdaq extends selloff as...
1294,5,1392047300843696130,Self Care For Covidhttps://t.co/X36WRH1PHV
1295,5,1392047276546199552,There are nopatients in University Hospital
1296,5,1392047263241801729,"Telangana announces 10-dayfrom tomorrow. Also,..."


6    342
3    307
5    289
1    176
4    109
2     75
Name: label, dtype: int64


In [102]:
#We Tokenize the tweet texts

from pyspark.ml.feature import  Tokenizer
from pyspark.ml.feature import CountVectorizer, HashingTF
from pyspark.ml.feature import  IDF
final_data = df2.withColumn("label", df2["label"].cast(IntegerType()))

tokenizer = Tokenizer(inputCol="tweet_text", outputCol="tweet_text_tokens")

data_words = tokenizer.transform(final_data)
data_words.show()

data_words_train, data_words_test = data_words.randomSplit([0.8,0.2],0)


data_words_df = data_words_test.toPandas()
display(data_words_df)

+-----+-------------------+--------------------+--------------------+
|label|           tweet_id|          tweet_text|   tweet_text_tokens|
+-----+-------------------+--------------------+--------------------+
|    1|1380150333767262211| bubbles under su...|[, bubbles, under...|
|    1|1380150113650274308|Nothing to see he...|[nothing, to, see...|
|    6|1380150618275389448|Well that was qui...|[well, that, was,...|
|    6|1380150554974904321|Good morning, Twi...|[good, morning,, ...|
|    6|1380150530526306314|Here for my secon...|[here, for, my, s...|
|    6|1380150486339366928|You don't have a ...|[you, don't, have...|
|    6|1380150434208358402|Because only cert...|[because, only, c...|
|    6|1380150386863013894|Ocugen: Potential...|[ocugen:, potenti...|
|    6|1380150373399339009|Proud to see thou...|[proud, to, see, ...|
|    5|1380150776299937795|The pandemic is n...|[the, pandemic, i...|
|    5|1380150698948575232|Tandon's next Emp...|[tandon's, next, ...|
|    5|1380150671123

,label,tweet_id,tweet_text,tweet_text_tokens
0,1,1382973402122747905,https://t.co/0u8EI0mOtRHuge potential !The dom...,"[https://t.co/0u8ei0motrhuge, potential, !the,..."
1,1,1382974202320326659,Theborder lets in abrigade each week.,"[theborder, lets, in, abrigade, each, week.]"
2,1,1382980968970092546,Fincrime experts warn compliance teams to step...,"[fincrime, experts, warn, compliance, teams, t..."
3,1,1382984000382193664,appeals toto lifton raw material exports http...,"[, appeals, toto, lifton, raw, material, expor..."
4,1,1382984666450423810,“One advisor to the Pentagon and National Secu...,"[“one, advisor, to, the, pentagon, and, nation..."
...,...,...,...,...
247,6,1391807754579312642,As a citizen ofwe have to support to our natio...,"[as, a, citizen, ofwe, have, to, support, to, ..."
248,6,1391807868807094275,"""Take a moment today to explore your patients’...","[""take, a, moment, today, to, explore, your, p..."
249,6,1392007999070240769,"INDIA'S DEVELOP NEW ANTI- DRUG, Read at: https...","[india's, develop, new, anti-, drug,, read, at..."
250,6,1392008006041227264,"Pfizer’s ex VP and chief scientist, Dr Mike Ye...","[pfizer’s, ex, vp, and, chief, scientist,, dr,..."


In [103]:
count = CountVectorizer(inputCol="tweet_text_tokens", outputCol="count_vectors")
count_model = count.fit(data_words_train)
data_count_vectors_train = count_model.transform(data_words_train)
data_count_vectors_train.show()

data_count_vectors_test = count_model.transform(data_words_test)
data_count_vectors_test.show()

data_count_vectors_df = data_count_vectors_train.toPandas()
display(data_count_vectors_df)

+-----+-------------------+--------------------+--------------------+--------------------+
|label|           tweet_id|          tweet_text|   tweet_text_tokens|       count_vectors|
+-----+-------------------+--------------------+--------------------+--------------------+
|    1|1380150113650274308|Nothing to see he...|[nothing, to, see...|(8083,[0,1,5,8,14...|
|    1|1380150333767262211| bubbles under su...|[, bubbles, under...|(8083,[5,6,7,81,2...|
|    1|1380150964167045120|court case agains...|[court, case, aga...|(8083,[0,4,9,14,3...|
|    1|1380150981040689156|“Will the species...|[“will, the, spec...|(8083,[0,1,4,7,8,...|
|    1|1380151008832188433|An Israeli planni...|[an, israeli, pla...|(8083,[0,3,5,9,37...|
|    1|1380151176994295810|260 CONCENTRATION...|[260, concentrati...|(8083,[3,38,122,1...|
|    1|1380151189199814656| will addresstoda...|[, will, addresst...|(8083,[6,18,583,1...|
|    1|1380151372369305603|📛“Do Chinese war...|[📛“do, chinese, ...|(8083,[0,1,2,3,16...|
|

,label,tweet_id,tweet_text,tweet_text_tokens,count_vectors
0,1,1380150113650274308,Nothing to see here...Five people killed ininc...,"[nothing, to, see, here...five, people, killed...","(1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, ..."
1,1,1380150333767262211,bubbles under surface.... Backsbut Some Call ...,"[, bubbles, under, surface...., backsbut, some...","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, ..."
2,1,1380150964167045120,court case against( was dismissed by twoalumni...,"[court, case, against(, was, dismissed, by, tw...","(1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
3,1,1380150981040689156,“Will the species survive? Will organized huma...,"[“will, the, species, survive?, will, organize...","(4.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, ..."
4,1,1380151008832188433,An Israeli planning commission advanced 540 ne...,"[an, israeli, planning, commission, advanced, ...","(3.0, 0.0, 0.0, 3.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...
1041,6,1392011017668816896,But authorities have ordered all foreign helpe...,"[but, authorities, have, ordered, all, foreign...","(1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1042,6,1392012958998294529,True colours of Maharastra Government.https://...,"[true, colours, of, maharastra, government.htt...","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1043,6,1392013096139431937,True colours of Maharastra Government.,"[, true, colours, of, maharastra, government.]","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
1044,6,1392013132625694721,government in Maharashtra was not giving perm...,"[, government, in, maharashtra, was, not, givi...","(2.0, 2.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, ..."


In [104]:
idf = IDF(inputCol="count_vectors", outputCol="features")
idf_model = idf.fit(data_count_vectors_train)
idf_data_train = idf_model.transform(data_count_vectors_train)
idf_data_test = idf_model.transform(data_count_vectors_test)

preprocessed_data_train = idf_data_train.select("label", "features")
preprocessed_data_train = preprocessed_data_train.withColumn("label", preprocessed_data_train["label"].cast(IntegerType()))
preprocessed_data_train.show()


preprocessed_data_test = idf_data_test.select("label", "features")
preprocessed_data_test = preprocessed_data_test.withColumn("label", preprocessed_data_test["label"].cast(IntegerType()))
preprocessed_data_test.show()


preproc_df = preprocessed_data_test.toPandas()
print(preproc_df)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    1|(8083,[0,1,5,8,14...|
|    1|(8083,[5,6,7,81,2...|
|    1|(8083,[0,4,9,14,3...|
|    1|(8083,[0,1,4,7,8,...|
|    1|(8083,[0,3,5,9,37...|
|    1|(8083,[3,38,122,1...|
|    1|(8083,[6,18,583,1...|
|    1|(8083,[0,1,2,3,16...|
|    1|(8083,[0,1,2,6,8,...|
|    1|(8083,[0,1,3,4,7,...|
|    1|(8083,[2,6,7,707,...|
|    1|(8083,[1,3,20,95,...|
|    1|(8083,[19,32,177,...|
|    1|(8083,[4,6,7,9,12...|
|    1|(8083,[0,1,6,7,13...|
|    1|(8083,[0,2,11,25,...|
|    1|(8083,[2,6,8,29,9...|
|    1|(8083,[7,429,641,...|
|    1|(8083,[2,6,7,9,10...|
|    1|(8083,[7,2165,340...|
+-----+--------------------+
only showing top 20 rows

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    1|(8083,[7,392,608,...|
|    1|(8083,[3,593,6702...|
|    1|(8083,[1,2,3,37,4...|
|    1|(8083,[6,602,852,...|
|    1|(8083,[0,1,4,7,8,...|
|    1|(8083,[2,14,51,83...|
|    1|(8083,[0,4

In [105]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import numpy as np

log_model = LogisticRegression(maxIter = 100)
#elasticnetPar = 0 : ridge , elasticnetPar = 1 : lasso  regpar = lambda 
grid = ParamGridBuilder().addGrid(log_model.regParam, np.linspace(0, 5,10)).addGrid(log_model.elasticNetParam, [1]).build()
cross_val = CrossValidator(estimator=log_model,
                          estimatorParamMaps=grid,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds= 2)  
cross_val = cross_val.fit(preprocessed_data_train)

In [106]:
best_model = cross_val.bestModel
print(best_model)

LogisticRegressionModel: uid=LogisticRegression_2c8453c66623, numClasses=7, numFeatures=8083


In [107]:
pred = best_model.transform(preprocessed_data_test)
pred.show()
pd_pred = pred.toPandas()
display(pd_pred)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|    1|(8083,[7,392,608,...|[-4.6434028763138...|[6.80209463201022...|       1.0|
|    1|(8083,[3,593,6702...|[-4.4068702493526...|[8.96468413469298...|       6.0|
|    1|(8083,[1,2,3,37,4...|[-4.7166611436049...|[3.21806047915081...|       5.0|
|    1|(8083,[6,602,852,...|[-4.4569796039494...|[2.86406528381061...|       6.0|
|    1|(8083,[0,1,4,7,8,...|[-5.1053583893374...|[7.64858004070547...|       3.0|
|    1|(8083,[2,14,51,83...|[-4.6506322944802...|[6.66117829484640...|       3.0|
|    1|(8083,[0,4,5,8,14...|[-4.8483122775307...|[1.15890956960282...|       1.0|
|    1|(8083,[1937,3377]...|[-4.4244826453837...|[1.51322755881268...|       1.0|
|    1|(8083,[0,1,2,3,6,...|[-4.9415509789541...|[4.64900121672200...|       5.0|
|    1|(8083,[0,

,label,features,rawPrediction,probability,prediction
0,1,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.54203815...","[-4.643402876313876, 9.556924860064235, -2.661...","[6.802094632010225e-07, 0.9994628286164692, 4....",1.0
1,1,"(0.0, 0.0, 0.0, 1.3153295415367912, 0.0, 0.0, ...","[-4.406870249352691, 0.5555169448958538, 0.805...","[0.000896468413469298, 0.1281363555345598, 0.1...",6.0
2,1,"(0.0, 0.8760419675215024, 1.2171119133913448, ...","[-4.716661143604972, -0.4438364100717759, -2.0...","[3.21806047915081e-05, 0.002308120114471481, 0...",5.0
3,1,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.3260630971799...","[-4.456979603949421, -0.19152991821640586, -0....","[0.00028640652838106146, 0.020391265271743064,...",6.0
4,1,"(1.6623828027123015, 0.8760419675215024, 0.0, ...","[-5.105358389337454, 2.520063943315958, -3.098...","[7.64858004070547e-06, 0.015676880833313936, 5...",3.0
...,...,...,...,...,...
247,6,"(0.0, 1.7520839350430049, 1.2171119133913448, ...","[-4.9442759060219315, 3.484070575327499, -0.79...","[8.904265730249052e-05, 0.40736384445312007, 0...",6.0
248,6,"(0.8311914013561508, 0.8760419675215024, 0.0, ...","[-4.840606593469324, 7.547532183541117, -3.128...","[4.149695512743595e-06, 0.995672570597141, 2.3...",1.0
249,6,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-4.501142288356601, 0.7155255750854221, 0.467...","[0.00040997050435918675, 0.07556534669006508, ...",6.0
250,6,"(0.0, 0.8760419675215024, 0.0, 0.0, 4.08809169...","[-4.77868152778076, 2.291579080266982, -3.1005...","[5.9723551051293636e-05, 0.07026204563613199, ...",5.0


In [111]:
pred.groupBy('label','prediction').count().sort('label','prediction').show(50)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       1.0|   15|
|    1|       3.0|    6|
|    1|       5.0|    4|
|    1|       6.0|   10|
|    2|       2.0|    4|
|    2|       3.0|    4|
|    2|       5.0|    2|
|    2|       6.0|    4|
|    3|       1.0|    1|
|    3|       2.0|    1|
|    3|       3.0|   28|
|    3|       4.0|    1|
|    3|       5.0|   12|
|    3|       6.0|   17|
|    4|       3.0|    4|
|    4|       4.0|    5|
|    4|       5.0|    5|
|    4|       6.0|    8|
|    5|       1.0|    1|
|    5|       3.0|    6|
|    5|       5.0|   24|
|    5|       6.0|   18|
|    6|       1.0|    1|
|    6|       3.0|    3|
|    6|       5.0|   25|
|    6|       6.0|   43|
+-----+----------+-----+



In [109]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

# Create both evaluators
evaluatorMulti = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")

# Make predicitons
predictionAndTarget = best_model.transform(preprocessed_data_test).select("label", "prediction")

# Get metrics
Accuracy = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "accuracy"})
WeightedF1 = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "weightedFMeasure"})
weightedPrecision = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "weightedPrecision"})
weightedRecall = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "weightedRecall"})

print("Weighted recall = %s" % weightedRecall)
print("Weighted precision = %s" % weightedPrecision)
print("Weighted F1 Score = %s" % WeightedF1)
print("Accuracy = %s" % Accuracy)

Weighted recall = 0.4722222222222222
Weighted precision = 0.5513274198568316
Weighted F1 Score = 0.4732995525919087
Accuracy = 0.4722222222222222


In [116]:
count_model.write().overwrite().save('countvectorizer')
idf_model.write().overwrite().save('tf-idf')
best_model.write().overwrite().save('log_regression')

In [80]:
from pyspark.streaming import StreamingContext
from pyspark.sql import Row
from pyspark.sql.functions import udf, struct, array, col, lit
from pyspark.sql.types import StringType
 

In [117]:
globals()['models_loaded'] = False
globals()['my_model'] = None

from pyspark.ml.tuning import CrossValidatorModel
from pyspark.ml.classification import LogisticRegressionModel
from  pyspark.ml.feature import CountVectorizerModel
from pyspark.ml.feature import  IDFModel 

    
def process(time, rdd):
    if rdd.isEmpty():
        return
    
  
    print("========= %s =========" % str(time))
    
    # Convert to data frame
    df = spark.read.json(rdd)
    df.show()
    
    # Load in the model if not yet loaded:
    if not globals()['models_loaded']:
        # load in your models here
        globals()['my_log_model'] = LogisticRegressionModel.load('log_regression')
        globals()['my_count_vectoriser'] = CountVectorizerModel.load('countvectorizer')
        globals()['my_tf_idf'] = IDFModel.load('tf-idf')        
        globals()['models_loaded'] = True
        
    # And then predict using the loaded model: 
    
    token = tokenizer.transform(df)
    count = my_count_vectoriser.transform(token)
    idf = my_tf_idf.transform(count)
    idf_select = idf.select("label", "features")
    
    predict = my_log_model.transform(idf_select)
    predict_names = predict.withColumn("prediction", when(predict.prediction == 1, '#biden')
                                 .when(predict.prediction == 2,'#inflation')
                    .when(predict.prediction == 3,'#china')
                    .when(predict.prediction == 4,"#stopasianhate")
                    .when(predict.prediction == 5,"#covid")
                    .when(predict.prediction == 6,"#vaccine")
                                 .when(predict.prediction.isNull() ,"")
                                 .otherwise(predict.prediction))
    predict_names.show() 



In [118]:
ssc = StreamingContext(sc, 10)

In [119]:
lines = ssc.socketTextStream("seppe.net", 7778)
lines.foreachRDD(process)

In [121]:
ssc_t = StreamingThread(ssc)
ssc_t.start()

========= 2021-05-14 15:02:20 =========
+------+-------------------+--------------------+
| label|           tweet_id|          tweet_text|
+------+-------------------+--------------------+
|#covid|1393187507764822018|Ola announced 'O2...|
+------+-------------------+--------------------+

+------+--------------------+--------------------+--------------------+----------+
| label|            features|       rawPrediction|         probability|prediction|
+------+--------------------+--------------------+--------------------+----------+
|#covid|(8083,[1,2,276,30...|[-4.5639633780221...|[1.54877837832899...|    #covid|
+------+--------------------+--------------------+--------------------+----------+

========= 2021-05-14 15:02:30 =========
+------+-------------------+--------------------+
| label|           tweet_id|          tweet_text|
+------+-------------------+--------------------+
|#covid|1393187480078209025|Today our milkman...|
+------+-------------------+--------------------+

+-

========= 2021-05-14 15:04:20 =========
+--------------+-------------------+--------------------+
|         label|           tweet_id|          tweet_text|
+--------------+-------------------+--------------------+
|        #china|1393188309153767425|@AbiyAhmedAli , #...|
|#stopasianhate|1393188702529142784|#███████ is organ...|
+--------------+-------------------+--------------------+

+--------------+--------------------+--------------------+--------------------+----------+
|         label|            features|       rawPrediction|         probability|prediction|
+--------------+--------------------+--------------------+--------------------+----------+
|        #china|(8083,[1,3,5,9,10...|[-4.8855625477133...|[5.84647223367698...|    #covid|
|#stopasianhate|(8083,[0,1,4,6,7,...|[-5.1211815930571...|[8.90859743630979...|  #vaccine|
+--------------+--------------------+--------------------+--------------------+----------+

========= 2021-05-14 15:04:40 =========
+------+----------------

========= 2021-05-14 15:06:50 =========
+--------+-------------------+--------------------+
|   label|           tweet_id|          tweet_text|
+--------+-------------------+--------------------+
|#vaccine|1393189575896584194|Just scheduled a ...|
+--------+-------------------+--------------------+

+--------+--------------------+--------------------+--------------------+----------+
|   label|            features|       rawPrediction|         probability|prediction|
+--------+--------------------+--------------------+--------------------+----------+
|#vaccine|(8083,[5,7,13,32,...|[-5.1707280786750...|[1.58308982043067...|  #vaccine|
+--------+--------------------+--------------------+--------------------+----------+

========= 2021-05-14 15:07:20 =========
+------+-------------------+--------------------+
| label|           tweet_id|          tweet_text|
+------+-------------------+--------------------+
|#china|1393189906407739395|“The Best Bible S...|
+------+-------------------+-----

In [122]:
ssc_t.stop()

----- Stopping... this may take a few seconds -----
